In this Notebook we implement the incremental forms of all SQL operators (and more) in DBSP as they were defined on subsection 4.2 of https://arxiv.org/pdf/2203.16684

There are far easier ways to go about this, but this is the most "proper".

In [1]:
from typing import Optional
from typing import TypeVar
from pydbsp.stream import LiftedGroupAdd, Stream, StreamHandle, step_until_fixpoint
from pydbsp.stream import BinaryOperator
from pydbsp.stream.operators.linear import LiftedStreamElimination, LiftedStreamIntroduction
from pydbsp.zset import ZSet
from pydbsp.zset.operators.unary import DeltaLiftedDeltaLiftedDistinct
from pydbsp.utils.stream import from_dict_into_singleton_stream

T = TypeVar("T")

class Union(BinaryOperator[Stream[ZSet[T]], Stream[ZSet[T]], Stream[ZSet[T]]]):
    """
    (SELECT * FROM I1)
    UNION
    (SELECT * FROM I2)
    """

    frontier_a: int
    frontier_b: int

    addition: LiftedGroupAdd[Stream[ZSet[T]]]
    distinct: DeltaLiftedDeltaLiftedDistinct[T]

    def set_input_a(self, stream_handle_a: StreamHandle[Stream[ZSet[T]]]) -> None:
        self.input_stream_handle_a = stream_handle_a

    def set_input_b(self, stream_handle_b: StreamHandle[Stream[ZSet[T]]]) -> None:
        self.input_stream_handle_b = stream_handle_b
        self.addition = LiftedGroupAdd(self.input_stream_handle_a, self.input_stream_handle_b)
        self.distinct = DeltaLiftedDeltaLiftedDistinct(self.addition.output_handle())

        self.output_stream = self.distinct.output()
        self.output_stream_handle = self.distinct.output_handle()

    def __init__(self, stream_a: Optional[StreamHandle[Stream[ZSet[T]]]], stream_b: Optional[StreamHandle[Stream[ZSet[T]]]]):
        self.frontier_a = 0
        self.frontier_b = 0

        if stream_a is not None:
            self.set_input_a(stream_a)
        if stream_b is not None:
            self.set_input_b(stream_b)

    def step(self) -> bool:
        current_a_timestamp = self.input_a().current_time()
        current_b_timestamp = self.input_b().current_time()

        if current_a_timestamp == self.frontier_a and current_b_timestamp == self.frontier_b:
            return True

        self.frontier_a += 1
        self.frontier_b += 1

        self.addition.step()

        return self.distinct.step()

# This function will create a Stream where each item of the dictionary will occupy a timestamp
# If { 1: 1, 2: 1, 3: 1 } is given, it will create a stream with 3 items:
# - { 1: 1 } at timestamp 0
# - { 2: 1 } at timestamp 1
# - { 3: -1 } at timestamp 2
I1 = from_dict_into_singleton_stream({ 1: 1, 2: 1, 3: -1 })
lifted_lifted_I1 = LiftedStreamIntroduction(I1)
step_until_fixpoint(lifted_lifted_I1)

# Notice how 3 here has a positive weight..
I2 = from_dict_into_singleton_stream({ 2: 1, 3: 1 })
lifted_lifted_I2 = LiftedStreamIntroduction(I2)
step_until_fixpoint(lifted_lifted_I2)

union = Union(lifted_lifted_I1.output_handle(), lifted_lifted_I2.output_handle())
step_until_fixpoint(union)

output = LiftedStreamElimination(union.output_handle())
step_until_fixpoint(output)

# The expected output is a stream with 3 timestamps, each outputting a "diff", that is, what "changes" should be applied to ensure that the "Union" computation remains correct.
print(output.output())

OrderedDict({1: {1: 1, 2: 1}, 2: {3: 1}, 3: {3: -1}})


In [2]:
from pydbsp.stream import UnaryOperator
from pydbsp.zset.operators.linear import LiftedLiftedProject
from pydbsp.zset.functions.linear import Projection as project, T, R

class Projection(UnaryOperator[Stream[ZSet[T]], Stream[ZSet[R]]]):
    """
    SELECT DISTINCT I.c
    FROM I
    """
    frontier: int
    stream: StreamHandle[Stream[ZSet[T]]]
    lifted_lifted_project: LiftedLiftedProject[T, R]
    distinct: DeltaLiftedDeltaLiftedDistinct[R]

    # This would require some dependent typing to be properly implemented.
    def __init__(self, stream: StreamHandle[Stream[ZSet[T]]], projection: project[T, R]):
        self.stream = stream
        self.frontier = 0
        self.lifted_lifted_project = LiftedLiftedProject(stream, projection)
        self.distinct = DeltaLiftedDeltaLiftedDistinct(self.lifted_lifted_project.output_handle())
        self.output_stream = self.distinct.output()
        self.output_stream_handle = self.distinct.output_handle()

    def step(self) -> bool:
        latest = self.lifted_lifted_project.input_a().current_time()

        if latest == self.frontier:
            return True

        self.frontier += 1

        self.lifted_lifted_project.step()

        return self.distinct.step()

I = from_dict_into_singleton_stream({ ("a", "b"): 1, ("c", "d"): 1, ("e", "f"): 1 })
lifted_lifted_I = LiftedStreamIntroduction(I)
step_until_fixpoint(lifted_lifted_I)

projection = Projection(lifted_lifted_I.output_handle(), lambda x: x[0])
step_until_fixpoint(projection)

output = LiftedStreamElimination(projection.output_handle())
step_until_fixpoint(output)

print(output.output())

OrderedDict({1: {'a': 1}, 2: {'c': 1}, 3: {'e': 1}})


In [3]:
from pydbsp.stream import UnaryOperator
from pydbsp.zset.operators.linear import LiftedLiftedSelect
from pydbsp.zset.functions.linear import Cmp, T

class Filtering(UnaryOperator[Stream[ZSet[T]], Stream[ZSet[T]]]):
    """
    SELECT DISTINCT * FROM I
    WHERE p(I.c)
    """
    frontier: int
    stream: StreamHandle[Stream[ZSet[T]]]
    lifted_lifted_selection: LiftedLiftedSelect[T]
    distinct: DeltaLiftedDeltaLiftedDistinct[T]

    def __init__(self, stream: StreamHandle[Stream[ZSet[T]]], selection: Cmp[T]):
        self.stream = stream
        self.frontier = 0
        self.lifted_lifted_selection = LiftedLiftedSelect(stream, selection)
        self.distinct = DeltaLiftedDeltaLiftedDistinct(self.lifted_lifted_selection.output_handle())
        self.output_stream = self.distinct.output()
        self.output_stream_handle = self.distinct.output_handle()

    def step(self) -> bool:
        latest = self.lifted_lifted_selection.input_a().current_time()

        if latest == self.frontier:
            return True

        self.frontier += 1

        self.lifted_lifted_selection.step()

        return self.distinct.step()

I = from_dict_into_singleton_stream({ ("a", "b"): 1, ("a", "d"): 1, ("e", "f"): 1 })
lifted_lifted_I = LiftedStreamIntroduction(I)
step_until_fixpoint(lifted_lifted_I)

filtering = Filtering(lifted_lifted_I.output_handle(), lambda x: x[0] == "a")
step_until_fixpoint(filtering)

output = LiftedStreamElimination(filtering.output_handle())
step_until_fixpoint(output)

print(output.output())

OrderedDict({1: {('a', 'b'): 1}, 2: {('a', 'd'): 1}})


In [4]:
class Selection(UnaryOperator[Stream[ZSet[T]], Stream[ZSet[R]]]):
    """
    SELECT DISTINCT f(I.c, ...)
    FROM I
    """
    frontier: int
    stream: StreamHandle[Stream[ZSet[T]]]
    lifted_lifted_project: LiftedLiftedProject[T, R]
    distinct: DeltaLiftedDeltaLiftedDistinct[R]

    def __init__(self, stream: StreamHandle[Stream[ZSet[T]]], selection: project[T, R]):
        self.stream = stream
        self.frontier = 0
        self.lifted_lifted_project = LiftedLiftedProject(stream, selection)
        self.distinct = DeltaLiftedDeltaLiftedDistinct(self.lifted_lifted_project.output_handle())
        self.output_stream = self.distinct.output()
        self.output_stream_handle = self.distinct.output_handle()

    
    def step(self) -> bool:
        latest = self.lifted_lifted_project.input_a().current_time()

        if latest == self.frontier:
            return True

        self.frontier += 1

        self.lifted_lifted_project.step()

        return self.distinct.step()

I = from_dict_into_singleton_stream({ (1, "b"): 1, (2, "d"): 1, (3, "f"): 1 })
lifted_lifted_I = LiftedStreamIntroduction(I)
step_until_fixpoint(lifted_lifted_I)

selection = Selection(lifted_lifted_I.output_handle(), lambda x: x[0] ** 2)
step_until_fixpoint(selection)

output = LiftedStreamElimination(selection.output_handle())
step_until_fixpoint(output)

print(output.output())


OrderedDict({1: {1: 1}, 2: {4: 1}, 3: {9: 1}})


In [5]:
from pydbsp.zset.operators.bilinear import DeltaLiftedDeltaLiftedJoin

class CartesianProduct(BinaryOperator[Stream[ZSet[T]], Stream[ZSet[R]], Stream[ZSet[tuple[T, R]]]]):
    """
    SELECT I1.*, I2.*
    FROM I1, I2
    """
    frontier_a: int
    frontier_b: int

    cartesian_product: DeltaLiftedDeltaLiftedJoin[T, R, tuple[T, R]]

    def set_input_a(self, stream_handle_a: StreamHandle[Stream[ZSet[T]]]) -> None:
        self.input_stream_handle_a = stream_handle_a

    def set_input_b(self, stream_handle_b: StreamHandle[Stream[ZSet[R]]]) -> None:
        self.input_stream_handle_b = stream_handle_b
        self.cartesian_product = DeltaLiftedDeltaLiftedJoin(self.input_stream_handle_a, self.input_stream_handle_b, lambda x, y: True, lambda x, y: (x, y))
        self.output_stream = self.cartesian_product.output()
        self.output_stream_handle = self.cartesian_product.output_handle()

    def __init__(self, stream_a: Optional[StreamHandle[Stream[ZSet[T]]]], stream_b: Optional[StreamHandle[Stream[ZSet[R]]]]):
        if stream_a is not None:
            self.set_input_a(stream_a)
        if stream_b is not None:
            self.set_input_b(stream_b)
    
    def step(self) -> bool:
        return self.cartesian_product.step()

I1 = from_dict_into_singleton_stream({ 1: 1, 2: 1, 3: 1 })
lifted_lifted_I1 = LiftedStreamIntroduction(I1)
step_until_fixpoint(lifted_lifted_I1)

I2 = from_dict_into_singleton_stream({ 1: 1, 2: 1, 3: 1 })
lifted_lifted_I2 = LiftedStreamIntroduction(I2)
step_until_fixpoint(lifted_lifted_I2)

cartesian_product = CartesianProduct(lifted_lifted_I1.output_handle(), lifted_lifted_I2.output_handle())
step_until_fixpoint(cartesian_product)

output = LiftedStreamElimination(cartesian_product.output_handle())
step_until_fixpoint(output)

print(output.output())


OrderedDict({1: {(1, 1): 1}, 2: {(2, 2): 1, (1, 2): 1, (2, 1): 1}, 3: {(3, 3): 1, (2, 3): 1, (1, 3): 1, (3, 2): 1, (3, 1): 1}})


In [6]:
from pydbsp.zset.operators.bilinear import JoinCmp

class Join(BinaryOperator[Stream[ZSet[T]], Stream[ZSet[R]], Stream[ZSet[tuple[T, R]]]]):
    """
    SELECT I1.*, I2.*
    FROM I1 JOIN I2
    ON I1.c1 = I2.c2
    """
    
    join: DeltaLiftedDeltaLiftedJoin[T, R, tuple[T, R]]
    on: JoinCmp[T, R]

    def set_input_a(self, stream_handle_a: StreamHandle[Stream[ZSet[T]]]) -> None:
        self.input_stream_handle_a = stream_handle_a

    def set_input_b(self, stream_handle_b: StreamHandle[Stream[ZSet[R]]]) -> None:
        self.input_stream_handle_b = stream_handle_b
        self.join = DeltaLiftedDeltaLiftedJoin(self.input_stream_handle_a, self.input_stream_handle_b, self.on, lambda x, y: (x, y))
        self.output_stream = self.join.output()
        self.output_stream_handle = self.join.output_handle()

    def __init__(self, stream_a: Optional[StreamHandle[Stream[ZSet[T]]]], stream_b: Optional[StreamHandle[Stream[ZSet[R]]]], on: JoinCmp[T, R]):
        self.on = on
        if stream_a is not None:
            self.set_input_a(stream_a)
        if stream_b is not None:
            self.set_input_b(stream_b)

    def step(self) -> bool:
        return self.join.step()

I1 = from_dict_into_singleton_stream({ ("a", "b"): 1, ("a", "d"): 1, ("e", "f"): 1 })
lifted_lifted_I1 = LiftedStreamIntroduction(I1)
step_until_fixpoint(lifted_lifted_I1)

I2 = from_dict_into_singleton_stream({ ("a", "b"): 1, ("c", "d"): 1, ("e", "g"): 1 })
lifted_lifted_I2 = LiftedStreamIntroduction(I2)
step_until_fixpoint(lifted_lifted_I2)

join = Join(lifted_lifted_I1.output_handle(), lifted_lifted_I2.output_handle(), lambda x, y: x[0] == y[0])
step_until_fixpoint(join)

output = LiftedStreamElimination(join.output_handle())
step_until_fixpoint(output)

print(output.output())

OrderedDict({1: {(('a', 'b'), ('a', 'b')): 1}, 2: {(('a', 'd'), ('a', 'b')): 1}, 3: {(('e', 'f'), ('e', 'g')): 1}})


In [7]:
class Intersection(BinaryOperator[Stream[ZSet[T]], Stream[ZSet[T]], Stream[ZSet[T]]]):
    """
    (SELECT * FROM I1)
    INTERSECT
    (SELECT * FROM I2)
    """
    
    intersection: DeltaLiftedDeltaLiftedJoin[T, T, T]

    def set_input_a(self, stream_handle_a: StreamHandle[Stream[ZSet[T]]]) -> None:
        self.input_stream_handle_a = stream_handle_a

    def set_input_b(self, stream_handle_b: StreamHandle[Stream[ZSet[T]]]) -> None:
        self.input_stream_handle_b = stream_handle_b
        self.intersection = DeltaLiftedDeltaLiftedJoin(self.input_stream_handle_a, self.input_stream_handle_b, lambda x, y: x == y, lambda x, y: x)
        self.output_stream = self.intersection.output()
        self.output_stream_handle = self.intersection.output_handle()
    
    def __init__(self, stream_a: Optional[StreamHandle[Stream[ZSet[T]]]], stream_b: Optional[StreamHandle[Stream[ZSet[T]]]]):
        if stream_a is not None:
            self.set_input_a(stream_a)
        if stream_b is not None:
            self.set_input_b(stream_b)

    def step(self) -> bool:
        return self.intersection.step()

I1 = from_dict_into_singleton_stream({ 1: 1, 2: 1, 3: 1 })
lifted_lifted_I1 = LiftedStreamIntroduction(I1)
step_until_fixpoint(lifted_lifted_I1)

I2 = from_dict_into_singleton_stream({ 1: 1, 2: 1, 3: -1 })
lifted_lifted_I2 = LiftedStreamIntroduction(I2)
step_until_fixpoint(lifted_lifted_I2)

intersection = Intersection(lifted_lifted_I1.output_handle(), lifted_lifted_I2.output_handle())
step_until_fixpoint(intersection)

output = LiftedStreamElimination(intersection.output_handle())
step_until_fixpoint(output)

print(output.output())


OrderedDict({1: {1: 1}, 2: {2: 1}, 3: {3: -1}})


In [8]:
from pydbsp.stream.operators.linear import LiftedGroupNegate

class Difference(BinaryOperator[Stream[ZSet[T]], Stream[ZSet[T]], Stream[ZSet[T]]]):
    """
    SELECT * FROM I1
    EXCEPT
    SELECT * FROM I2
    """

    frontier_a: int
    frontier_b: int

    negation: LiftedGroupNegate[Stream[ZSet[T]]]
    addition: LiftedGroupAdd[Stream[ZSet[T]]]
    distinct: DeltaLiftedDeltaLiftedDistinct[T]

    def set_input_a(self, stream_handle_a: StreamHandle[Stream[ZSet[T]]]) -> None:
        self.input_stream_handle_a = stream_handle_a

    def set_input_b(self, stream_handle_b: StreamHandle[Stream[ZSet[T]]]) -> None:
        self.input_stream_handle_b = stream_handle_b

        self.negation = LiftedGroupNegate(self.input_stream_handle_b)
        self.addition = LiftedGroupAdd(self.input_stream_handle_a, self.negation.output_handle())
        self.distinct = DeltaLiftedDeltaLiftedDistinct(self.addition.output_handle())
        self.output_stream = self.distinct.output()
        self.output_stream_handle = self.distinct.output_handle()

    def __init__(self, stream_a: Optional[StreamHandle[Stream[ZSet[T]]]], stream_b: Optional[StreamHandle[Stream[ZSet[T]]]]):
        if stream_a is not None:
            self.set_input_a(stream_a)
        if stream_b is not None:
            self.set_input_b(stream_b)
        
        self.frontier_a = 0
        self.frontier_b = 0
    
    def step(self) -> bool:
        current_a_timestamp = self.input_a().current_time()
        current_b_timestamp = self.input_b().current_time()

        if current_a_timestamp == self.frontier_a and current_b_timestamp == self.frontier_b:
            return True

        self.frontier_a += 1
        self.frontier_b += 1

        self.negation.step()

        self.addition.step()

        return self.distinct.step()

I1 = from_dict_into_singleton_stream({ 1: 1, 2: 1, 3: 1 })
lifted_lifted_I1 = LiftedStreamIntroduction(I1)
step_until_fixpoint(lifted_lifted_I1)

I2 = from_dict_into_singleton_stream({ 1: 1, 2: 1, 4: 1 })
lifted_lifted_I2 = LiftedStreamIntroduction(I2)
step_until_fixpoint(lifted_lifted_I2)

difference = Difference(lifted_lifted_I1.output_handle(), lifted_lifted_I2.output_handle())
step_until_fixpoint(difference)

output = LiftedStreamElimination(difference.output_handle())
step_until_fixpoint(output)

print(output.output())

OrderedDict({3: {3: 1}})


In [9]:
from pydbsp.indexed_zset import Indexer, I
from pydbsp.stream import Lift1, step_until_fixpoint_and_return
from typing import Callable

from pydbsp.stream.operators.linear import Differentiate, Integrate, LiftedIntegrate
from pydbsp.zset import ZSetAddition

# We assume that the aggregation function is linear, that is, f(a + b) = f(a) + f(b)
# Mind you, Lift1 and Lift2 functions are for direct manipulation of ZSets, not of their values.
Aggregation = Callable[[ZSet[tuple[I, ZSet[T]]]], ZSet[tuple[I, R]]]

def group_zset(zset: ZSet[T], by: Indexer[T, I]) -> ZSet[tuple[I, ZSet[T]]]:
    grouping: dict[I, ZSet[T]] = {}
    for k, v in zset.items():
        group = by(k)

        if group not in grouping:
            new_zset = ZSet({k: v})
            grouping[group] = new_zset
        else:
            zset_addition: ZSetAddition[T] = ZSetAddition()
            grouping[group] = zset_addition.add(a=grouping[group], b=ZSet({k: v}))

    return ZSet({(group, zset_grouped): 1 for group, zset_grouped in grouping.items()})

class LiftedAggregate(Lift1[ZSet[T], ZSet[tuple[I, R]]]):
    def __init__(self, stream: StreamHandle[ZSet[T]], by: Indexer[T, I], aggregation: Aggregation[I, T, R]):
        super().__init__(stream, lambda zset: aggregation(group_zset(zset, by)), None)

class LiftedLiftedAggregate(Lift1[Stream[ZSet[T]], Stream[ZSet[tuple[I, R]]]]):
    def __init__(self, stream: StreamHandle[Stream[ZSet[T]]], by: Indexer[T, I], aggregation: Aggregation[I, T, R]):
        super().__init__(stream, lambda sp: step_until_fixpoint_and_return(LiftedAggregate(StreamHandle(lambda: sp), by, aggregation)), None)

def group_by_fst[K, V](input: tuple[K, V]) -> K:
    return input[0]

# This is a proper weighted ZSet sum.
def zset_sum(input: ZSet[tuple[I, ZSet[int]]]) -> ZSet[tuple[I, int]]:
    output_dict: dict[I, int] = {}
    for (group, zset), _ in input.items():
        for k, v in zset.items():
            if group not in output_dict:
                output_dict[group] = k[1] * v
            else:
                output_dict[group] += (k[1] * v)
    
    return ZSet({(group_fst, v): 1 for group_fst, v in output_dict.items()})

I1 = from_dict_into_singleton_stream({ ("a", 2): 1, ("a", 3): 1, ("b", 1): 1 })
lifted_lifted_I1 = LiftedStreamIntroduction(I1)
step_until_fixpoint(lifted_lifted_I1)

non_incremental_agg = LiftedLiftedAggregate(lifted_lifted_I1.output_handle(), group_by_fst, zset_sum)
step_until_fixpoint(non_incremental_agg)

output = LiftedStreamElimination(non_incremental_agg.output_handle())
step_until_fixpoint(output)

print(output.output())

# We will now implement an iterative version of what is above. 
# By applying stream elimination and differentiating its output, we get an incremental version.

class GroupByThenAgg(UnaryOperator[ZSet[T], ZSet[tuple[I, R]]]):
    """
    SELECT I.c1, AGG(I.c2)
    FROM I
    GROUP BY I.c1
    """

    frontier: int

    integrated_stream: Integrate[Stream[ZSet[T]]]
    lift_integrated_stream: LiftedIntegrate[ZSet[T]]
    lifted_lifted_aggregate: LiftedLiftedAggregate[T, I, R]

    def set_input(self, stream_handle: StreamHandle[Stream[ZSet[T]]]) -> None:
        self.input_stream_handle = stream_handle
        self.integrated_stream = Integrate(stream_handle)
        self.lift_integrated_stream = LiftedIntegrate(self.integrated_stream.output_handle())
        self.lifted_lifted_aggregate = LiftedLiftedAggregate(self.lift_integrated_stream.output_handle(), self.by, self.aggregation)

        self.output_stream = self.lifted_lifted_aggregate.output()
        self.output_stream_handle = self.lifted_lifted_aggregate.output_handle()
    
    def __init__(self, stream: StreamHandle[Stream[ZSet[T]]], by: Indexer[T, I], aggregation: Aggregation[I, T, R]):
        self.by = by
        self.aggregation = aggregation
        self.frontier = 0
        self.set_input(stream)
    
    def step(self) -> bool:
        latest = self.input_stream_handle.get().current_time()

        if latest == self.frontier:
            return True

        self.frontier += 1

        self.integrated_stream.step()
        self.lift_integrated_stream.step()
        
        return self.lifted_lifted_aggregate.step()

I2 = from_dict_into_singleton_stream({ ("a", 2): 1, ("a", 3): 1, ("b", 1): 1, ("a", 5): -1 })
lifted_lifted_I2 = LiftedStreamIntroduction(I2)
step_until_fixpoint(lifted_lifted_I2)

incremental_agg = GroupByThenAgg(lifted_lifted_I2.output_handle(), group_by_fst, zset_sum)
step_until_fixpoint(incremental_agg)

output = LiftedStreamElimination(incremental_agg.output_handle())
step_until_fixpoint(output)

differentiated_output = Differentiate(output.output_handle())
step_until_fixpoint(differentiated_output)

print(differentiated_output.output())    

OrderedDict({1: {('a', 2): 1}, 2: {('a', 3): 1}, 3: {('b', 1): 1}})
OrderedDict({1: {('a', 2): 1}, 2: {('a', 5): 1, ('a', 2): -1}, 3: {('b', 1): 1}, 4: {('a', 0): 1, ('a', 5): -1}})


In [ ]:
def zset_max(input: ZSet[tuple[I, ZSet[int]]]) -> ZSet[tuple[I, int]]:
    output_dict: dict[I, int] = {}
    for (group, zset), _ in input.items():
        for k, v in zset.items():
            if group not in output_dict:
                output_dict[group] = k[1] * v
            else:
                output_dict[group] = max(output_dict[group], k[1] * v)
    
    return ZSet({(group_fst, v): 1 for group_fst, v in output_dict.items()})

I3 = from_dict_into_singleton_stream({ ("a", 2): 1, ("a", 3): 1, ("b", 1): 1, ("a", 5): -1 })
for i in I3.get():
    print(i)
# Notice that we add **one** more element to the stream, and this element has a negative value i.e we are retracting ("a", 3). It is expected
# That the maximum value of "a" will then be 2, therefore this should be reflected in the output.
I3.get().send(ZSet({("a", 3): -1}))
lifted_lifted_I3 = LiftedStreamIntroduction(I3)
step_until_fixpoint(lifted_lifted_I3)

incremental_max = GroupByThenAgg(lifted_lifted_I3.output_handle(), group_by_fst, zset_max)
step_until_fixpoint(incremental_max)

output = LiftedStreamElimination(incremental_max.output_handle())
step_until_fixpoint(output)

differentiated_output = Differentiate(output.output_handle())
step_until_fixpoint(differentiated_output)

print(differentiated_output.output())  

{}
{('a', 2): 1}
{('a', 3): 1}
{('b', 1): 1}
{('a', 5): -1}
OrderedDict({1: {('a', 2): 1}, 2: {('a', 3): 1, ('a', 2): -1}, 3: {('b', 1): 1}, 5: {('a', 2): 1, ('a', 3): -1}})
